In [1]:
from pyviper.pp import _viper_similarity
import inspect
import pyviper
import numpy as np
import pandas as pd
from scipy.stats import norm
import scanpy as sc
import matplotlib.pyplot as plt 
from itertools import combinations

/Users/rcassius/miniforge3/envs/pyviper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pyviper
print(pyviper.__file__)


/Users/rcassius/projects/pyviper/pyviper/__init__.py


In [3]:
resources_dir = "/Users/rcassius/projects/pyviper/tests/resources"

In [4]:
from os.path import join

In [6]:
gene_expr_path = "/Users/rcassius/projects/pyviper/tests/resources/ges.csv"
gene_expr_signature = pd.read_csv(gene_expr_path, sep=",", index_col=0)
gene_expr_signature

,T11_AAACCTGTCCTGTACC,T6_GACGCGTTCATGTCCC,T21_TGCTACCAGGGTGTTG,T13_GAGGTGACACCACGTG,T2_AGGGAGTAGGCTCAGA,T6_CATGCCTTCGGTCTAA,T21_CGAGAAGTCACTTATC,T7_GTATCTTAGAAGAAGC,T5_CTAGCCTGTGTCGCTG,T23_CTGAAGTTCCAACCAA,...,T21_TACGGATAGTTGAGAT,T5_GACCTGGAGTCATCCA,T15_CCTATTAAGTGCGATG,T2_CGACTTCAGTTGTAGA,T13_CGTGTCTGTATCGCAT,T6_TGACTTTAGTACGATA,T6_TTGAACGCAAGGCTCC,T24_TGCCCTAAGCCACGCT,T2_GCTTCCAGTAGAGGAA,T13_TCGGGACAGACTTGAA
RP11-206L10.2,-0.030106,-0.043220,-0.037076,-0.064336,-0.035180,-0.059154,-0.052739,-0.030590,-0.018762,-0.065870,...,-0.084270,-0.097196,-0.035755,-0.035338,-0.098253,-0.041776,-0.096049,-0.031701,-0.026148,-0.074726
RP11-206L10.9,-0.048024,-0.065633,-0.057379,-0.093971,-0.054833,-0.087028,-0.078420,-0.048673,-0.032836,-0.096025,...,-0.120556,-0.137642,-0.055606,-0.055046,-0.139032,-0.063693,-0.136131,-0.050163,-0.042718,-0.107858
SAMD11,-0.053621,-0.072847,-0.063835,-0.103773,-0.061056,-0.096199,-0.086805,-0.054330,-0.037047,-0.106013,...,-0.132736,-0.151311,-0.061899,-0.061287,-0.152820,-0.070729,-0.149670,-0.055957,-0.047830,-0.118910
PLEKHN1,-0.145538,-0.185591,-0.166844,-0.249476,-0.161052,-0.233902,-0.214519,-0.147019,-0.110853,-0.254073,...,-0.308529,2.121368,-0.162811,-0.161535,-0.348946,-0.181190,-0.342638,-0.150416,-0.133432,-0.280445
HES4,-0.819920,-0.070299,1.164534,-1.224043,0.193488,0.901816,0.441105,0.369486,0.957019,-1.239576,...,-0.938442,1.642698,2.307027,0.187800,-0.353236,-0.026433,0.899302,0.324864,0.563274,0.297872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P2RY14,-0.016508,-0.026891,-0.022031,-0.043566,-0.020530,-0.039476,-0.034411,-0.016892,-0.007488,-0.044778,...,-0.059318,-0.069563,-0.020985,-0.020655,-0.070402,-0.025749,-0.068652,-0.017773,-0.013366,-0.051772
P2RY13,-0.039399,-0.055535,-0.047972,-0.081517,-0.045639,-0.075147,-0.067254,-0.039995,-0.025479,-0.083402,...,-0.105944,-0.121686,-0.046347,-0.045834,-0.122969,9.410099,-0.120293,-0.041360,-0.034537,-0.094268
AADAC,-0.076620,-0.098689,-0.088344,-0.134174,-0.085153,-0.125489,-0.114711,-0.077434,-0.057611,-0.136742,...,-0.167333,-0.188535,-0.086122,-0.085419,-0.190255,-0.096258,-0.186664,-0.079301,-0.069975,-0.151517
SUCNR1,-0.035747,-0.050768,-0.043728,-0.074965,-0.041556,-0.069030,-0.061679,-0.036302,-0.022781,-0.076722,...,-0.097755,-0.112477,-0.042216,-0.041737,-0.113678,-0.049113,-0.111172,-0.037573,-0.031220,-0.086855


In [6]:
# Set the HPC file path
# gene_expr_path = "/home/friva/TCGA_DEG/DEG-BRCA.tsv"
gene_expr_path = "/Users/rcassius/projects/pyviper/tests/resources/ges.csv"
# Read the data
# gene_expr_signature = pd.read_csv(gene_expr_path, sep="\t", index_col=0)
gene_expr_signature = pd.read_csv(gene_expr_path, sep=",", index_col=0).T
# Convert to AnnData object
gene_expr_signature = sc.AnnData(gene_expr_signature)
# Inspect
gene_expr_signature

AnnData object with n_obs × n_vars = 100 × 1000

In [7]:
network_path = "/Users/rcassius/projects/pyviper/tests/resources/test_net1.tsv"
network = pd.read_csv(network_path, delimiter="\t")
network

,regulator,target,mor,likelihood
0,AATF,IK,0.084484,0.999854
1,AATF,JAK1,0.066589,0.999709
2,AATF,CBR4,0.011550,0.999563
3,AATF,CCPG1,0.025028,0.999417
4,AATF,CHD2,-0.010843,0.999271
...,...,...,...,...
54624,CDH1,FGD3,0.101185,0.923385
54625,CDH1,POLE4,-0.011756,0.923242
54626,CDH1,PRR13,-0.090628,0.923099
54627,CDH1,MRPL13,0.008079,0.922956


In [8]:

network_path = "/home/friva/TCGA_networks/net-prunned-BRCA_test.tsv"
network_path = "/Users/rcassius/projects/pyviper/tests/resources/test_net1.tsv"

network = pd.read_csv(network_path, delimiter="\t")

# Drop the first column if it's just an index
# network = network.drop(network.columns[0], axis=1)

# network_interactome = pyviper.Interactome('TCGA_BRCA', network)  # convert to class Interactome
network_interactome = pyviper.Interactome('test_net1', network)  # convert to class Interactome
network_interactome.size()  # show number of regulons in the Interactome
network_interactome.net_table.head()

,regulator,target,mor,likelihood
0,AATF,IK,0.084484,0.999854
1,AATF,JAK1,0.066589,0.999709
2,AATF,CBR4,0.011550,0.999563
3,AATF,CCPG1,0.025028,0.999417
4,AATF,CHD2,-0.010843,0.999271


In [9]:
# keep targets present in expression
network_interactome.filter_targets(gene_expr_signature.var_names)
# prune to exactly 50 targets per TF (drop TFs with <50)
network_pruned = network_interactome.copy()
network_pruned.prune(max_targets=25, eliminate=True)
# network_pruned.prune(max_targets=50, eliminate=False)

Removed 52565 targets.
Removed 1439 targets.
Removed 75 regulators.


/Users/rcassius/projects/pyviper/pyviper/interactome.py:655: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pruned_df = sorted_df.groupby('regulator', group_keys=False).apply(lambda x: x.iloc[:max_targets])


In [10]:
gene_expr_signature.X.shape

(100, 1000)

In [11]:
network_pruned.net_table

,regulator,target,mor,likelihood
38,ACAD8,SLC44A3,0.077846,0.998677
39,ACAD8,MYO1B,0.000731,0.996788
40,ACAD8,RP11-277L2.5,-0.079591,0.996221
41,ACAD8,TSGA10,-0.021371,0.989040
42,ACAD8,MGLL,-0.004368,0.980726
...,...,...,...,...
1882,CDCA7,AC011747.4,-0.029880,0.911239
1883,CDCA7,AC116035.1,0.022639,0.893578
1884,CDCA7,PLCL2,-0.063177,0.892431
1885,CDCA7,TACSTD2,0.041609,0.891055


In [12]:
ProtAct_aREA = pyviper.viper(
    gex_data=gene_expr_signature, # gene expression signature
    interactome=network_pruned, # gene regulatory network
    enrichment = "area",
    min_targets=0,
    eset_filter=True,
    njobs=1,
    verbose=False
)

In [13]:
ProtAct_aREA.to_df()

,ACAD8,AEBP2,AGO1,AKIRIN2,APP,ARID3A,ARNT,ASXL3,ATAD2,ATF2,...,BNC2,BRD8,BRF2,BTF3,BTG1,CAMTA2,CBFA2T2,CC2D1A,CCNK,CDCA7
T11_AAACCTGTCCTGTACC,0.634558,-0.101231,-1.184505,-0.008175,-3.924742,-0.042705,0.999103,-0.007888,-0.031060,-0.965028,...,-0.040753,2.191276,-0.008887,-1.690514,-2.242315,0.860045,-0.382058,-2.370249,1.209191,0.018058
T6_GACGCGTTCATGTCCC,-1.144576,-0.129430,-0.906499,1.048647,-2.848629,0.430151,-0.992621,0.347979,0.018564,-1.155773,...,-0.083598,1.119768,0.020616,-1.228266,4.250925,-0.076245,-0.165809,-2.175829,-0.810720,0.050493
T21_TGCTACCAGGGTGTTG,-0.572921,-0.104154,1.383405,-0.035823,-3.402809,-0.015386,-1.049427,-0.019404,0.061305,-1.580801,...,0.008562,-1.353077,0.624964,0.987875,-4.020088,0.013796,-0.074459,1.722050,-0.870688,-0.526119
T13_GAGGTGACACCACGTG,0.033500,-0.043519,1.264250,0.014348,-2.231847,-0.027925,-1.344375,-0.020533,-0.033658,-2.277062,...,0.010430,-2.154860,-0.030948,-0.993476,-3.706919,0.108679,-0.512614,2.733434,1.726322,0.024398
T2_AGGGAGTAGGCTCAGA,-0.484677,-0.149509,2.238583,-0.300237,-3.788477,-0.883459,1.997197,-0.012000,0.003119,-0.976650,...,-0.056847,-0.853918,-0.001997,-1.296712,-4.104563,-0.008344,0.375902,-1.902468,-0.654970,0.080236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T6_TGACTTTAGTACGATA,-1.182156,0.264845,1.365864,0.691023,2.044844,-0.010434,-1.319140,0.024107,-0.062153,-1.721586,...,-0.078678,-1.805501,0.041417,1.733964,3.743625,0.022624,-0.521310,-1.358552,1.390177,0.038420
T6_TTGAACGCAAGGCTCC,0.419020,2.049427,0.961018,-0.238156,1.268144,-0.007433,-0.860373,0.120857,-0.032453,1.592355,...,-0.001071,-3.002068,0.048553,-2.288648,2.239098,0.057738,1.350723,-0.412511,-1.998287,0.033271
T24_TGCCCTAAGCCACGCT,-1.147180,0.036991,-1.073046,-0.046391,3.814164,-0.010270,2.095735,0.016780,0.083403,-1.951035,...,-0.065645,-0.970646,0.013839,-1.557747,3.547992,-0.034669,-0.590645,2.616269,-1.031033,0.048146
T2_GCTTCCAGTAGAGGAA,-1.572179,-0.070758,0.991536,-0.013121,-3.613705,-0.008715,0.955799,0.012308,0.039031,-2.179393,...,-0.085108,1.444266,-0.044123,1.183207,-3.901528,-0.033210,-0.135235,-1.871869,-0.326918,0.018533


In [1]:
sim = _viper_similarity(
    ProtAct_aREA, nn= None , ws=(4.0, 2.0), method="two.sided",
    random_state=0, store_in_adata=True, key_added="viper_similarity"
)

NameError: name '_viper_similarity' is not defined

In [60]:
sim.to_csv(join(resources_dir, "viper_similarity_py_output_two_sided.csv"))

In [61]:
simg = _viper_similarity(
    ProtAct_aREA, nn= None , ws=(4.0, 2.0), method="greater",
    random_state=0, store_in_adata=True, key_added="viper_similarity"
)

In [62]:
simg.to_csv(join(resources_dir, "viper_similarity_py_output_greater.csv"))

In [63]:
siml = _viper_similarity(
    ProtAct_aREA, nn= None , ws=(4.0, 2.0), method="less",
    random_state=0, store_in_adata=True, key_added="viper_similarity"
)

In [64]:
# siml.to_csv("VIPER_similarity_PY_output_less_github.csv")
siml.to_csv(join(resources_dir, "viper_similarity_py_output_less.csv"))

In [65]:
sim50 = _viper_similarity(
    ProtAct_aREA, nn= 50 , ws=(4.0, 2.0), method="less",
    random_state=0, store_in_adata=True, key_added="viper_similarity"
)

In [66]:
sim50.to_csv(join(resources_dir, "viper_similarity_py_output_nn_50.csv"))